In [ ]:
# pip install -q transformers
from transformers import AutoModelForCausalLM, AutoTokenizer


checkpoint = "output_dir/checkpoint-7800"#"bigscience/bloomz-3b" #"bigscience/bloom-7b1"# 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).half().cuda()



In [ ]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\n{instruction}\n\n### Response:"
    ),
}

In [ ]:
from typing import Optional
def generate_input(instruction:Optional[str]= None, input_str:Optional[str] = None) -> str:
    if input_str is None:
        return PROMPT_DICT['prompt_no_input'].format_map({'instruction':instruction})
    else:
        return PROMPT_DICT['prompt_input'].format_map({'instruction':instruction, 'input':input_str})
    

In [ ]:
for i in range(10):
    print("*"*80)

    inputs = tokenizer.encode(generate_input(instruction="写一篇关于水杯的文章"), return_tensors="pt")
    inputs = inputs.to(model.device)
    outputs = model.generate(inputs,num_beams=2,
                            max_new_tokens=1024,
                            do_sample=True, 
                            top_k=10,
                            penalty_alpha=0.6,
                            temperature=0.8,
                            repetition_penalty=1.2)
    print(tokenizer.decode(outputs[0]))